<a href="https://colab.research.google.com/github/Ali-Felfel/NLP/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text


In [ ]:
data_train = pd.read_csv("training_set.txt", delimiter = "\t",encoding='utf-8')
data_test = pd.read_csv("dev_set.txt", delimiter = "\t",encoding='utf-8')

In [ ]:
data_testforreal = pd.read_csv("test_set.txt", delimiter = "\t",encoding='utf-8')


In [ ]:
X_train = data_train['sentence'].tolist()
X_test = data_test['sentence'].tolist()

y_train = data_train['emotion'].tolist()
y_test = data_test['emotion'].tolist()

data = data_train.append(data_test, ignore_index=True)

# class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']
class_names = ['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy','Sadness','Surprise','Trust']


print('size of training set: %s' % (len(data_train['sentence'])))
print('size of validation set: %s' % (len(data_test['sentence'])))
print(data.emotion.value_counts())

data.head(10)

size of training set: 14000
size of validation set: 1000
1    3210
2    2299
8    2253
5    1567
4    1546
6    1471
3    1420
7    1234
Name: emotion, dtype: int64


,sentence,emotion
0,I'm too old to be traded in .,6
1,Mother said you could always tell a lady by he...,8
2,I always said I'd leave off when the time came .,6
3,He'll be safe with me .,2
4,Lay off .,1
5,You tell him to take care of you .,8
6,I hope so .,4
7,Why do you want to shut me out in the cold lik...,6
8,"You taught me a lesson , man .",8
9,I'll do everything I can to make a success of ...,2


In [ ]:
# encoding = {
#     'joy': 0,
#     'sadness': 1,
#     'fear': 2,
#     'anger': 3,
#     'neutral': 4
# }
encoding = {1 : "Anger", 2 : "Anticipation", 3: "Disgust", 4 : "Fear", 5 : "Joy",6 : "Sadness",7 : "Surprise",8 : "Trust"}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

In [ ]:
np.unique(y_train)

array(['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness',
       'Surprise', 'Trust'], dtype='<U12')

In [ ]:
np.unique(y_test)

array(['Anger', 'Anticipation', 'Disgust', 'Fear', 'Joy', 'Sadness',
       'Surprise', 'Trust'], dtype='<U12')

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:738: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  "class_names argument was ignored, as they were extracted from string labels in dataset"


task: text classification


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)



Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
2334/2334 [==============================] - 1300s 548ms/step - loss: 1.6090 - accuracy: 0.4067 - val_loss: 1.3825 - val_accuracy: 0.4880
Epoch 2/3
2334/2334 [==============================] - 1276s 547ms/step - loss: 1.2395 - accuracy: 0.5576 - val_loss: 1.3939 - val_accuracy: 0.5130
Epoch 3/3
2334/2334 [==============================] - 1274s 546ms/step - loss: 0.7523 - accuracy: 0.7433 - val_loss: 1.5546 - val_accuracy: 0.5120


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

       Anger       0.53      0.58      0.55       211
Anticipation       0.57      0.49      0.53       170
     Disgust       0.41      0.35      0.38        77
        Fear       0.47      0.53      0.50       104
         Joy       0.56      0.56      0.56        97
     Sadness       0.46      0.51      0.48        87
    Surprise       0.42      0.41      0.41        96
       Trust       0.56      0.55      0.56       158

    accuracy                           0.51      1000
   macro avg       0.50      0.50      0.50      1000
weighted avg       0.51      0.51      0.51      1000



array([[122,  11,  20,  17,   3,  15,  13,  10],
       [ 21,  84,   4,  18,  11,   2,  15,  15],
       [ 26,   2,  27,   7,   2,   4,   5,   4],
       [ 12,   6,   2,  55,   2,  14,   4,   9],
       [  4,   7,   2,   1,  54,   4,   8,  17],
       [ 12,   7,   7,   6,   3,  44,   3,   5],
       [ 18,   9,   3,   9,   5,   6,  39,   7],
       [ 16,  21,   1,   5,  16,   7,   5,  87]])